In [1]:
%%time

import requests
import datetime
import pathlib
import apimoex
import aiomoex
import pandas as pd



Wall time: 2.31 s


In [170]:
#другой способ

# import pandas_datareader.data as web
# f = web.DataReader('SBER', 'moex', start='2001-01-01', end='2027-01-31')
# f.sort_index(inplace=True, ascending=True)
# z = f[['CLOSE', 'OPEN', 'HIGH', 'LOW', 'VOLUME', 'SECID', 'VALUE', 'BOARDID']]
# z[z['BOARDID'] == 'TQBR']

In [59]:
df_all[['Date', 'Open', 'High','Low','Close', 'Volume']]

,Date,Open,High,Low,Close,Volume
352,2006-01-23 00:00:00,239.50000,239.500000,218.490000,228.750000,1.127977e+10
353,2006-01-30 00:00:00,231.04000,231.070000,215.000000,221.010000,1.069375e+10
354,2006-02-06 00:00:00,221.63000,224.800000,215.500000,222.000000,8.782785e+09
355,2006-02-13 00:00:00,221.98000,227.330000,217.930000,226.500000,8.661725e+09
356,2006-02-20 00:00:00,227.57000,230.160000,224.110000,227.380000,7.748495e+09
...,...,...,...,...,...,...
390,2020-08-17 00:00:00,0.03785,0.038100,0.036075,0.036360,5.106386e+09
391,2020-08-24 00:00:00,0.03646,0.036660,0.035360,0.035950,5.285553e+09
392,2020-08-31 00:00:00,0.03606,0.036195,0.034805,0.035335,4.952319e+09
393,2020-09-07 00:00:00,0.03525,0.035485,0.034500,0.035420,3.285066e+09


In [2]:
%%time

#board = 'TQBR'
board  = '1min'
df_all = pd.DataFrame()

with open("E:/Host/Data/Database/TICK___.txt", "r") as TICKs:
    TICKs = [line.rstrip() for line in TICKs]
#    TICKs = 'GAZP'
pathlib.Path("E:/Host/Data/Database/{}".format(board)).mkdir(parents=True, exist_ok=True) #создаем дир

with requests.Session() as session:
    for TICK in TICKs:

        data = apimoex.get_market_candles(session, TICK, interval=1, start = '2001-01-01', end = '2028-01-01')

        df = pd.DataFrame(data)
        display(df.columns)
        df.columns = ['Date', 'Open', 'Close', 'High','Low', 'Volume']
        df = df[['Date', 'Open', 'High','Low','Close', 'Volume']]
        df.sort_values(by = 'Date', inplace = True, ascending=True)
        df.to_csv("E:/Host/Data/Database/{}/{}.csv".format(board,TICK), index=False) #капаем сюды
#         df['Symbol'] = TICK
#         display(df['Date'][0])
#         df_all = df_all.append(df)

        
#df_all.to_csv("E:/Host/Data/Database/{}/{}.csv".format(board,'all'), index=False) #капаем сюды
#df

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Index(['begin', 'open', 'close', 'high', 'low', 'value'], dtype='object')

Wall time: 1h 9min 23s


In [22]:
df_all.to_csv('df_all_10min.csv')

In [81]:
TICKs

['GAZP', 'GMKN', 'LKOH', 'ROSN', 'SBER', 'SBERP', 'VTBR']

In [2]:
%%time

#РАЗОВАЯ ОБЬЕДИНЯЕМ ФИНАМОВСКИЕ ДО 2012 ГОДА И apimoex 
dfff_all = pd.DataFrame()
for pp in ['GAZP', 'GMKN', 'LKOH', 'ROSN', 'SBER', 'VTBR']: 
    #1block
    dff = pd.read_csv('E:/Host/Data/Database/10min/' + str(pp) + '_.csv', parse_dates=[['<DATE>', '<TIME>']])
    dff.columns   = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    dff['Volume'] = dff['Volume']*dff['Close']
    dff['Symbol'] = pp
    
    #1.5block
    dff06 = pd.read_csv('E:/Host/Data/Database/10min/' + str(pp) + '__.csv', parse_dates=[['<DATE>', '<TIME>']])
    dff06.columns   = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    dff06['Volume'] = dff06['Volume']*dff06['Close']
    dff06['Symbol'] = pp
    
    #2block
    dfff = pd.read_csv('E:/Host/Data/Database/10min/' + str(pp) + '.csv', parse_dates=['Date'])
    dfff['Symbol'] = pp
#    display(dff[dff['Date']>='2011-12-08 10:00:00'])
#    display(dfff)
    dfff = dfff.append(dff)
    dfff = dfff.append(dff06)
#    display(dfff.shape)
    dfff.drop_duplicates(['Date', 'Symbol'], inplace = True)
#    display(dfff.shape)
    dfff.sort_values(by = ['Date'], inplace=True)
#    display(dfff)
    dfff_all = dfff_all.append(dfff)
dfff_all.to_csv('E:/Host/Data/Database/10min/2006_to_now.csv')    

Wall time: 13.2 s


In [10]:
dfff_all.groupby([ dfff_all['Date'].dt.year]).count()

,Date,Open,High,Low,Close,Volume,Symbol
Date,,,,,,,
2006,54134,54134,54134,54134,54134,54134,54134
2007,62704,62704,62704,62704,62704,62704,62704
2008,64830,64830,64830,64830,64830,64830,64830
2009,74299,74299,74299,74299,74299,74299,74299
2010,74400,74400,74400,74400,74400,74400,74400
2011,75810,75810,75810,75810,75810,75810,75810
2012,81026,81026,81026,81026,81026,81026,81026
2013,78960,78960,78960,78960,78960,78960,78960
2014,77914,77914,77914,77914,77914,77914,77914


In [83]:
dfff_all

,Date,Open,High,Low,Close,Volume,Symbol
0,2009-01-11 10:30:00,115.010000,115.010000,111.770000,112.740000,4.651402e+08,GAZP
1,2009-01-11 10:40:00,112.740000,113.400000,112.700000,113.050000,3.043665e+08,GAZP
2,2009-01-11 10:50:00,113.070000,113.790000,113.050000,113.400000,1.857546e+08,GAZP
3,2009-01-11 11:00:00,113.400000,113.800000,113.300000,113.800000,1.247881e+08,GAZP
4,2009-01-11 11:10:00,113.800000,113.900000,113.000000,113.250000,1.373856e+08,GAZP
...,...,...,...,...,...,...,...
119747,2020-09-18 11:00:00,0.035580,0.035665,0.035565,0.035570,2.255754e+07,VTBR
119748,2020-09-18 11:10:00,0.035570,0.035600,0.035530,0.035565,1.062869e+07,VTBR
119749,2020-09-18 11:20:00,0.035560,0.035565,0.035505,0.035555,7.170856e+06,VTBR
119750,2020-09-18 11:30:00,0.035545,0.035555,0.035520,0.035535,3.007063e+06,VTBR


In [34]:
df_all.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol'], dtype='object')

In [227]:
fff = part_1.append(df)

In [229]:
fff.to_csv('1min_blue.csv')

In [130]:
with requests.Session() as session:
    data = apimoex.get_market_candles(session, 'LKOH', interval=1, start = '2019-01-01', end = '2020-01-01')

In [131]:
pd.options.display.max_rows=100
df = pd.DataFrame(data)
df['begin'] = pd.to_datetime(df['begin'])
df.sort_values(by = 'begin', inplace=True, ascending=True)
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df

,Date,Open,High,Low,Close,Volume
0,2019-01-03 09:59:00,4984.0,4984.0,4984.0,4984.0,2681392.0
1,2019-01-03 10:00:00,4982.0,4950.0,4982.0,4926.5,39778187.0
2,2019-01-03 10:01:00,4950.0,4929.0,4953.5,4928.0,26338952.0
3,2019-01-03 10:02:00,4934.0,4932.0,4935.0,4928.5,9039022.5
4,2019-01-03 10:03:00,4932.0,4931.0,4934.0,4930.0,12112146.0
...,...,...,...,...,...,...
131520,2019-12-30 18:45:00,6169.0,6169.0,6169.0,6169.0,594080869.0
131521,2019-12-30 18:46:00,6169.0,6169.0,6169.0,6169.0,6452774.0
131522,2019-12-30 18:47:00,6169.0,6169.0,6169.0,6169.0,4793313.0
131523,2019-12-30 18:48:00,6169.0,6169.0,6169.0,6169.0,314619.0


In [135]:
pi

,count
,Close
Date,
2019-01-03,521
2019-01-04,519
2019-01-08,524
2019-01-09,523
2019-01-10,524
...,...
2019-12-24,526
2019-12-25,523


In [140]:
pi = pd.pivot_table(df, index=df['Date'].dt.date, values=['Close'], aggfunc=['count'])
#pd.unique(pi)
pi.describe()
pi.columns = ['count']
pi.sort_values(by = ['count'])

,count
Date,
2019-04-19,465
2019-07-23,495
2019-07-19,504
2019-07-22,506
2019-05-06,507
...,...
2019-04-23,526
2019-09-19,526
2019-09-13,526


In [69]:
df['begin'] = pd.to_datetime(df['begin'])

In [79]:
df.sort_values(by = 'begin', inplace=True, ascending=True)
df

,begin,open,close,high,low,value
92,2007-05-01 00:00:00,0.156400,0.241500,0.241500,0.156400,2.333857e+06
93,2007-09-01 00:00:00,0.107600,0.114900,0.126100,0.103700,3.433177e+10
94,2007-10-01 00:00:00,0.114200,0.119100,0.121200,0.110400,3.930202e+10
95,2007-10-01 00:00:00,0.119100,0.119300,0.119400,0.117200,2.224088e+09
96,2007-11-01 00:00:00,0.120000,0.112500,0.122400,0.105200,2.972693e+10
...,...,...,...,...,...,...
87,2020-05-01 00:00:00,0.034500,0.036310,0.036310,0.033915,1.944071e+10
88,2020-06-01 00:00:00,0.036420,0.035050,0.037980,0.034845,2.356729e+10
89,2020-07-01 00:00:00,0.035130,0.038755,0.039515,0.034180,3.244853e+10
90,2020-08-01 00:00:00,0.038835,0.035375,0.040725,0.035055,3.748719e+10


In [62]:
df = pd.DataFrame(data)


,begin,open,close,high,low,value
0,2013-02-28 10:00:00,0.05573,0.05590,0.05620,0.05573,24924184.8
1,2013-02-28 10:01:00,0.05588,0.05578,0.05590,0.05570,7267919.8
2,2013-02-28 10:02:00,0.05577,0.05577,0.05580,0.05573,4956947.6
3,2013-02-28 10:03:00,0.05574,0.05580,0.05580,0.05572,7277862.9
4,2013-02-28 10:04:00,0.05582,0.05586,0.05590,0.05582,5168863.7
...,...,...,...,...,...,...
1148003,2013-02-27 18:40:00,0.05500,0.05501,0.05502,0.05500,13188634.6
1148004,2013-02-27 18:41:00,0.05501,0.05508,0.05509,0.05500,14008288.8
1148005,2013-02-27 18:42:00,0.05508,0.05515,0.05515,0.05508,2815631.9
1148006,2013-02-27 18:43:00,0.05513,0.05514,0.05514,0.05505,8389471.4


In [52]:
apimoex.get_market_candle_borders

<function apimoex.requests.get_market_candle_borders(session: requests.sessions.Session, security: str, market: str = 'shares', engine: str = 'stock') -> List[Dict[str, Union[str, int, float]]]>

In [53]:
apimoex.get_market_candles

<function apimoex.requests.get_market_candles(session: requests.sessions.Session, security: str, interval: int = 24, start: Union[str, NoneType] = None, end: Union[str, NoneType] = None, columns: Union[Tuple[str, ...], NoneType] = ('begin', 'open', 'close', 'high', 'low', 'value'), market: str = 'shares', engine: str = 'stock') -> List[Dict[str, Union[str, int, float]]]>